In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [29]:
df = pd.read_csv('datasets/SatisfactionClients/cdiscount2.csv')
df_test = pd.read_csv('datasets/SatisfactionClients/amazon_test1.csv')
df.sample(3)

,note,commentaire
67891,5,prix assez bas appels téléphoniques plus propo...
12642,4,très bon qualité prix souvent livraisons rapides
18663,5,rapidité livraison petite remise sympa frais p...


In [30]:
df = df[df['commentaire'].isna()==False]

In [31]:
df.isna().sum()

note           0
commentaire    0
dtype: int64

In [32]:
# séparation de la variable cible et des variables explicatives
X = df['commentaire']
y = df['note']

In [33]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [34]:
# conversion des chaines de caratères en tokens numériques
# on ne prendra que les éléments répétés au moins 3 fois avec min_df=3

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, min_df=3)
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

In [7]:
np.save('datasets/SatisfactionClientsvector_matrix', X_train)

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=50)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(clf.score(X_test, y_test), end='\n\n')

cr = classification_report(y_test, y_pred)
print(cr)

cm = pd.crosstab(y_test, y_pred, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test)

for i in cm_test.index:
    for j in cm_test.columns :
        if (i==j and cm_test.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6888323069635244

              precision    recall  f1-score   support

           1       0.66      0.81      0.73      2038
           2       0.12      0.00      0.01       498
           3       0.34      0.13      0.18      1053
           4       0.45      0.21      0.29      3458
           5       0.74      0.93      0.82      9841

    accuracy                           0.69     16888
   macro avg       0.46      0.42      0.41     16888
weighted avg       0.63      0.69      0.64     16888



predictions,1,2,3,4,5
données réelles,,,,,
1,1641,1,68,69,259
2,252,2,53,67,124
3,257,9,133,233,421
4,179,4,104,738,2433
5,156,0,30,536,9119


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,1,3,4,5
données réelles,,,,
1,19,0,0,1
2,19,1,0,0
3,17,1,0,2
4,12,0,1,7
5,11,0,1,8


predictions,1,3,4,5
données réelles,,,,
1,0.95,0.00,0.00,0.05
2,0.95,0.05,0.00,0.00
3,0.85,0.05,0.00,0.10
4,0.60,0.00,0.05,0.35
5,0.55,0.00,0.05,0.40


les prédictions correctes sont supérieures à 50% pour 1 étoile(s) avec 95.0 %


In [36]:
from joblib import dump
dump(clf, 'datasets/SatisfactionClients/model_rf_tfidf.joblib')

['datasets/SatisfactionClients/model_rf_tfidf.joblib']

### Evaluation sur 2 sentiments : négatif et positif

* Le DataSet cdiscount a été retraité en renommant, les notes 1 et 2 en 0, les notes 4 et 5 en 1 et en supprimant les notes 3
* Le but étant d'isoler de manière moins complexe les sentiments négatifs et les sentiments positifs et ainsi améliorer la qualité du résultat prédit

In [37]:
df_0_1 = pd.read_csv('datasets/SatisfactionClients/cdiscount_0_1.csv')

In [38]:
df_0_1 = df_0_1[df_0_1['commentaire'].isna()==False]

In [39]:
# séparation de la variable cible et des variables explicatives
X1 = df_0_1['commentaire']
y1 = df_0_1['note']

In [40]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, shuffle=True)

In [41]:
X_train1 = vectorizer.transform(X_train1).todense()
X_test1 = vectorizer.transform(X_test1).todense()

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf_0_1 = RandomForestClassifier(n_estimators=50)

clf_0_1.fit(X_train1, y_train1)

y_pred_0_1 = clf_0_1.predict(X_test1)

print(clf_0_1.score(X_test1, y_test1), end='\n\n')

cr = classification_report(y_test1, y_pred_0_1)
print(cr)

cm = pd.crosstab(y_test1, y_pred_0_1, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf_0_1.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test2 = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test2)

for i in cm_test2.index:
    for j in cm_test2.columns :
        if (i==j and cm_test2.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test2.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.9411428933849179

              precision    recall  f1-score   support

           0       0.88      0.73      0.80      2547
           1       0.95      0.98      0.97     13220

    accuracy                           0.94     15767
   macro avg       0.92      0.86      0.88     15767
weighted avg       0.94      0.94      0.94     15767



predictions,0,1
données réelles,,
0,1864,683
1,245,12975


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,0,1
données réelles,,
1,19,1
2,17,3
3,13,7
4,8,12
5,9,11


predictions,0,1
données réelles,,
1,0.95,0.05
2,0.85,0.15
3,0.65,0.35
4,0.40,0.60
5,0.45,0.55


In [43]:
from joblib import dump
dump(clf_0_1, 'datasets/SatisfactionClients/model_rf_0_1_tfidf.joblib')

['datasets/SatisfactionClients/model_rf_0_1_tfidf.joblib']